In [87]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import shap
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_score, recall_score, f1_score
import xgboost

font = {'weight' : 'bold',
        'size'   : 14}
matplotlib.rc('font', **font)

In [88]:
# Model evaluation function

def modelEvaluation(X_test, y_test, model):
    y_pred = model.predict(X_test)
    print("        Accuracy: {:.3f}".format(model.score(X_test, y_test)))
    print(" Precision score: {:.3f}".format(precision_score(y_test, y_pred)))
    print("    Recall score: {:.3f}".format(recall_score(y_test, y_pred)))
    print("        F1 score: {:.3f}".format(f1_score(y_test, y_pred, average='weighted')))
    print()

In [89]:
# Custom progress callback function

def custom_progress_callback(estimator, params, mean_test_score, std_test_score):
    print(f"Hyperparameters: {params}")
    print(f"Mean Test Score: {mean_test_score}")
    print(f"Standard Deviation: {std_test_score}")
    print("-------------------------------")

In [ ]:
# Data loading

db = pd.read_csv("data.csv")
reacdb = db[(db['Melting']==1)]

reacdb = reacdb.sample(frac=1).reset_index(drop=True)
y = reacdb['Congruency']
X = reacdb.loc[:, 'Composition':'Angle sdev_dif'].astype('float')
X = pd.DataFrame(StandardScaler().fit(X).transform(X), columns = X.columns)

print(X.shape)

feature_names = X.columns

In [ ]:
# Data verification

print(f"Target binary compounds: {(db[db['CommonPair']==1].shape[0])}")
print(f"Melting system: {(db[db['Melting']==1].shape[0])}")
print(f"Coungruent melting compound: {(db[db['Congruency']==1].shape[0])}")
print(f"Application compounds: {(db[db['CommonPair']==0].shape[0])}")

In [ ]:
# Train, test set split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(y_train.shape)

In [ ]:
# Hyperparameter Tuning

param_test = {
    'max_depth':range(3,11,3),
    'min_child_weight':range(1,11,2),
    'gamma':[i/10.0 for i in range(0,6)],
    'subsample':[i/10.0 for i in range(6,10)],
    'colsample_bytree':[i/10.0 for i in range(6,10)],
    'n_estimators':[10, 100, 1000]
}

gsearch = GridSearchCV(estimator = xgboost.XGBClassifier(
    tree_method='gpu_hist',
    gpu_id=0, 
    use_label_encoder = False, 
    eval_metric='mlogloss',
    learning_rate = 0.1,
    n_estimators = 10,
    max_depth = 5,
    min_child_weight = 1,
    gamma = 0,
    subsample = 0.8,
    colsample_bytree = 0.8),
        param_grid = param_test,
        scoring = 'accuracy',
        n_jobs = 8,
        cv = 5, 
        verbose = 2)

gsearch.fit(X_train, y_train)

# gsearch.cv_results_, 
gsearch.best_params_, gsearch.best_score_

In [ ]:
# Model Fitting

model = xgboost.XGBClassifier(
    objective='binary:logistic',
    tree_method='gpu_hist', 
    gpu_id=0, 
    use_label_encoder = False, 
    eval_metric='auc',
    learning_rate = 0.3,
    n_estimators = 2000,
    max_depth = 6,
    min_child_weight = 5,
    gamma = 0.5,
    subsample = 0.9,
    colsample_bytree = 0.9)

model.fit(X_train, y_train, eval_set = [(X_test, y_test)], verbose=False)

In [ ]:
# Model evaluation and shap summary plot

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_train)
shap.summary_plot(shap_values, X_train, max_display=20, feature_names=feature_names)

modelEvaluation(X_test, y_test, model)

In [ ]:
# Extract top 10 features

importance = np.average(np.abs(shap_values),0)

sorted_indices = importance.argsort()[::-1]  # Sort indices in descending order
top_10_indices = sorted_indices[:10]  # Select the top 10 indices

top_10_features = X_train.iloc[0,top_10_indices].index  # Assuming X_train is your feature matrix

print(top_10_features)

In [ ]:
# Training with top 10 features

X_reduced_train = X_train.iloc[:, top_10_indices]
X_reduced_test = X_test.iloc[:, top_10_indices]

model.fit(X_reduced_train, y_train, eval_set = [(X_reduced_test, y_test)], verbose=False)

In [ ]:
# Model evaluation and shap summary plot with top 10 features

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_reduced_train)
shap.summary_plot(shap_values, X_reduced_train, max_display=10, feature_names=top_10_features)

modelEvaluation(X_reduced_test, y_test, model)

In [ ]:
y_pred = model.predict(X_reduced_test)

In [ ]:
y_test

In [146]:
compoundName = db['name']
compoundList = compoundName[y_test.index].values
predictionResult = pd.DataFrame(list(zip(compoundList, y_test, y_pred)),columns=['Name', 'Truth', 'Prediction'] ).sort_values('Name')

In [148]:
predictionResult.to_csv("predictionResult.csv")